In [1]:
from datasets import load_dataset

#直接使用我处理好的数据集
dataset = load_dataset(path='lansinuote/diffusion.4.text_to_image',
                       split='train')

dataset, dataset[0]

Using custom data configuration lansinuote--diffusion.4.text_to_image-c3c926d45403802a
Found cached dataset parquet (/root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--diffusion.4.text_to_image-c3c926d45403802a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


(Dataset({
     features: ['image', 'input_ids'],
     num_rows: 833
 }),
 {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1280x1280>,
  'input_ids': [49406,
   320,
   3610,
   539,
   320,
   1901,
   9528,
   593,
   736,
   3095,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407,
   49407]})

In [2]:
import torchvision

#数据增强
compose = torchvision.transforms.Compose([
    torchvision.transforms.Resize(
        512, interpolation=torchvision.transforms.InterpolationMode.BILINEAR),
    torchvision.transforms.CenterCrop(512),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5], [0.5]),
])


def f(data):
    image = [compose(i) for i in data['image']]
    return {'image': image, 'input_ids': data['input_ids']}


dataset = dataset.with_transform(f)

dataset, dataset[0]

(Dataset({
     features: ['image', 'input_ids'],
     num_rows: 833
 }),
 {'image': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],
  
          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],
  
          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]]]),
  'input_ids': [49406,
   320,
   3610,
   539,
   320,
   1901,
   9528,
   593,
  

In [3]:
import torch


def collate_fn(datas):
    image = torch.stack([i['image'] for i in datas], dim=0)
    input_ids = torch.LongTensor([i['input_ids'] for i in datas])

    return {'image': image, 'input_ids': input_ids}


loader = torch.utils.data.DataLoader(dataset,
                                     batch_size=1,
                                     shuffle=True,
                                     collate_fn=collate_fn)

len(loader), next(iter(loader))

(833,
 {'image': tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            ...,
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.]],
  
           [[1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            ...,
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.]],
  
           [[1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            ...,
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.]]]]),
  'input_ids': tensor([[49406,   320,  1579,   537,   736, 11781,  4668,   593,  1449,  3095,
           49407, 49407, 49407, 4940

In [4]:
from transformers.models.clip.modeling_clip import CLIPTextModel
from diffusers import AutoencoderKL, UNet2DConditionModel

checkpoint = 'runwayml/stable-diffusion-v1-5'

encoder = CLIPTextModel.from_pretrained(checkpoint, subfolder='text_encoder')
vae = AutoencoderKL.from_pretrained(checkpoint, subfolder='vae')
unet = UNet2DConditionModel.from_pretrained(checkpoint, subfolder='unet')

vae.requires_grad_(False)
encoder.requires_grad_(False)
unet.requires_grad_(False)


def print_model_size(name, model):
    print(name, sum(i.numel() for i in model.parameters()) / 10000)


print_model_size('encoder', encoder)
print_model_size('vae', vae)
print_model_size('unet', unet)

encoder 12306.048
vae 8365.3863
unet 85952.0964


In [5]:
#这个类不想测了,总之就是在做注意力的调整
from diffusers.models.attention_processor import LoRAAttnProcessor


def set_processors():
    processors = {}

    #遍历unet的所有层,找出所有有set_processor属性的层,每一个都组装成lora层
    for name in unet.attn_processors.keys():

        #768 = unet.config.cross_attention_dim
        cross_attention_dim = 768
        if name.endswith('attn1.processor'):
            cross_attention_dim = None

        if name.startswith('mid_block'):
            hidden_size = unet.config.block_out_channels[-1]

        if name.startswith('up_blocks'):
            #取层名字中的第一个数字
            #p_blocks.1.attentions.0.transformer_blocks.0.attn1.processor -> 1
            block_id = int(name[10])
            hidden_size = list(reversed(
                unet.config.block_out_channels))[block_id]

        if name.startswith('down_blocks'):
            #取层名字中的第一个数字
            #down_blocks.2.attentions.0.transformer_blocks.0.attn1.processor -> 2
            block_id = int(name[12])
            hidden_size = unet.config.block_out_channels[block_id]

        processors[name] = LoRAAttnProcessor(hidden_size, cross_attention_dim)

        print(name, hidden_size, cross_attention_dim)

    #把上面组装好的字典,设置到unet的层当中
    unet.set_attn_processor(processors)


set_processors()

down_blocks.0.attentions.0.transformer_blocks.0.attn1.processor 320 None
down_blocks.0.attentions.0.transformer_blocks.0.attn2.processor 320 768
down_blocks.0.attentions.1.transformer_blocks.0.attn1.processor 320 None
down_blocks.0.attentions.1.transformer_blocks.0.attn2.processor 320 768
down_blocks.1.attentions.0.transformer_blocks.0.attn1.processor 640 None
down_blocks.1.attentions.0.transformer_blocks.0.attn2.processor 640 768
down_blocks.1.attentions.1.transformer_blocks.0.attn1.processor 640 None
down_blocks.1.attentions.1.transformer_blocks.0.attn2.processor 640 768
down_blocks.2.attentions.0.transformer_blocks.0.attn1.processor 1280 None
down_blocks.2.attentions.0.transformer_blocks.0.attn2.processor 1280 768
down_blocks.2.attentions.1.transformer_blocks.0.attn1.processor 1280 None
down_blocks.2.attentions.1.transformer_blocks.0.attn2.processor 1280 768
up_blocks.1.attentions.0.transformer_blocks.0.attn1.processor 1280 None
up_blocks.1.attentions.0.transformer_blocks.0.attn2.pr

In [6]:
lora_layers = torch.nn.ModuleList(unet.attn_processors.values())

len(lora_layers)

32

In [7]:
from diffusers import DDPMScheduler
from transformers import AutoTokenizer
from diffusers.optimization import get_scheduler

tokenizer = AutoTokenizer.from_pretrained(checkpoint,
                                          subfolder='tokenizer',
                                          use_fast=False)

scheduler = DDPMScheduler.from_pretrained(checkpoint, subfolder='scheduler')

optimizer = torch.optim.AdamW(lora_layers.parameters(),
                              lr=1e-4,
                              betas=(0.9, 0.999),
                              weight_decay=0.01,
                              eps=1e-8)

scheduler_lr = get_scheduler('cosine',
                             optimizer=optimizer,
                             num_warmup_steps=0,
                             num_training_steps=len(loader) * 400 // 4)

criterion = torch.nn.MSELoss()

tokenizer, scheduler, optimizer, scheduler_lr, criterion

(CLIPTokenizer(name_or_path='runwayml/stable-diffusion-v1-5', vocab_size=49408, model_max_length=77, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<|endoftext|>'}),
 DDPMScheduler {
   "_class_name": "DDPMScheduler",
   "_diffusers_version": "0.15.1",
   "beta_end": 0.012,
   "beta_schedule": "scaled_linear",
   "beta_start": 0.00085,
   "clip_sample": false,
   "clip_sample_range": 1.0,
   "dynamic_thresholding_ratio": 0.995,
   "num_train_timesteps": 1000,
   "prediction_type": "epsilon",
   "sample_max_value": 1.0,
   "set_alpha_to_one": false,
   "skip_prk_steps": true,
   "steps_offset": 1,
   "thresholding": false,
   "t

In [8]:
def get_loss(data):
    device = data['input_ids'].device

    #编码文字
    #[1, 77] -> [1, 77, 768]
    out_encoder = encoder(data['input_ids'])[0]

    #vae计算特征图
    #[1, 3, 512, 512] -> [1, 4, 64, 64]
    out_vae = vae.encode(data['image']).latent_dist.sample()

    #0.18215 = vae.config.scaling_factor
    out_vae = out_vae * 0.18215

    #随机噪声
    #[1, 4, 64, 64]
    noise = torch.randn_like(out_vae)

    #随机噪声步
    #1000 = scheduler.config.num_train_timesteps
    #1 = b
    noise_step = torch.randint(0, 1000, (1, ), device=device).long()

    #添加噪声
    #[1, 4, 64, 64]
    out_vae_noise = scheduler.add_noise(out_vae, noise, noise_step)

    #unet从噪声图中把噪声计算出来
    #[1, 4, 64, 64],[1, 77, 768] -> [1, 4, 64, 64]
    out_unet = unet(out_vae_noise, noise_step, out_encoder).sample

    return criterion(out_unet, noise)


get_loss({
    'image': torch.randn(1, 3, 512, 512),
    'input_ids': torch.arange(77).reshape(1, 77)
})

tensor(0.6869, grad_fn=<MseLossBackward0>)

In [9]:
from diffusers import DiffusionPipeline


def train():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    unet.to(device)
    vae.to(device)
    encoder.to(device)
    unet.train()

    loss_sum = 0
    for epoch in range(150):
        for i, data in enumerate(loader):
            data['image'] = data['image'].to(device)
            data['input_ids'] = data['input_ids'].to(device)
            loss = get_loss(data) / 4
            loss.backward()

            if (epoch * len(loader) + i) % 4 == 0:
                torch.nn.utils.clip_grad_norm_(lora_layers.parameters(), 1.0)
                optimizer.step()
                optimizer.zero_grad()
                scheduler_lr.step()

            loss_sum += loss.item()

        if epoch % 5 == 0:
            print(epoch, loss_sum, optimizer.param_groups[-1]['lr'])
            loss_sum = 0

    unet.save_attn_procs('./save')


train()

0 10.703415064956062 9.99984467546043e-05
5 56.16534663550556 9.994444932466895e-05
10 52.10641818065778 9.981347814466988e-05
15 54.88103978987783 9.96057350657239e-05
20 53.18660080563859 9.932123063252573e-05
25 53.15705438374425 9.896094932338045e-05
30 51.69287981011439 9.852521002186447e-05
35 54.403384188131895 9.801468428384716e-05
40 54.496449730562745 9.742956212449453e-05
45 52.372736920719035 9.67718682576756e-05
50 53.141422546003014 9.604209018238184e-05
55 52.44126057237736 9.524135262330098e-05
60 53.340201130195055 9.437002038726624e-05
65 52.79343696354772 9.343110856853554e-05
70 52.16929884609999 9.242526127767233e-05
75 51.336487896653125 9.135402871372808e-05
80 52.107932627550326 9.021794149244833e-05
85 53.66056731872959 8.902093084685268e-05
90 53.038139166950714 8.776378163893685e-05
95 52.564585573622026 8.644843137107059e-05
100 52.97269177176349 8.5075563402391e-05
105 52.06748179736314 8.364992830467555e-05
110 52.05431841494283 8.2172432365445e-05
115 51.